In [31]:
import matplotlib.pyplot as plt
import requests
import urllib.request
from IPython.core.debugger import Tracer

url_form = "http://thegradcafe.com/survey/index.php?q=u%2A&t=a&pp=250&o=d&p={0}"
DATA_DIR = './WebScraped_data/html/'

if __name__ == '__main__':
    for i in range(1691, 1948):
        url = url_form.format(i)
        handle = urllib.request.urlopen(url)
        html = handle.read()
        html = html.decode('utf8')
        #r = requests.get(url)
        fname = "{data_dir}/{page}.html".format(data_dir=DATA_DIR,page=str(i))
        with open(fname, 'wb') as f:
            f.write(html.encode('UTF-8'))
        print("getting {0}...".format(i))

getting 1691...
getting 1692...
getting 1693...
getting 1694...
getting 1695...
getting 1696...
getting 1697...
getting 1698...
getting 1699...
getting 1700...
getting 1701...
getting 1702...
getting 1703...
getting 1704...
getting 1705...
getting 1706...
getting 1707...
getting 1708...
getting 1709...
getting 1710...
getting 1711...
getting 1712...
getting 1713...
getting 1714...
getting 1715...
getting 1716...
getting 1717...
getting 1718...
getting 1719...
getting 1720...
getting 1721...
getting 1722...
getting 1723...
getting 1724...
getting 1725...
getting 1726...
getting 1727...
getting 1728...
getting 1729...
getting 1730...
getting 1731...
getting 1732...
getting 1733...
getting 1734...
getting 1735...
getting 1736...
getting 1737...
getting 1738...
getting 1739...
getting 1740...
getting 1741...
getting 1742...
getting 1743...
getting 1744...
getting 1745...
getting 1746...
getting 1747...
getting 1748...
getting 1749...
getting 1750...
getting 1751...
getting 1752...
getting 

In [36]:
import datetime
import re
import sys
import pandas
from IPython.core.debugger import Tracer; debug_here = Tracer()
from bs4 import BeautifulSoup

PROGS = [
    ('Computer Engineering', 'ECE'),
    ('Computer Enginnerin', 'ECE'),
    ('Electrical', 'ECE'),
    ('ECE', 'ECE'),
    ('Computer Sc', 'CS'),
    ('Computer  Sc', 'CS'),
    ('Computer Sicen', 'CS'),
    ('Computer Sien', 'CS'),
    ('Computer S Cience', 'CS'),
    ('Computer,', 'CS'),
    ('Computers,', 'CS'),
    ('ComputerScience', 'CS'),
    ('Human Computer Interaction', 'HCI'),
    ('Human-Computer Interaction', 'HCI'),
    ('Human-computer Interaction', 'HCI'),
    ('software engineering', 'CS'),
    ('Embedded', 'ECE'),
    ('Computer Eng', 'ECE'),
    ('Computer Vision', 'CS'),
    ('Information', 'IS'),
    ('Infomation', 'IS'), ]


DEGREE = [
    (' MFA', 'MFA'),
    (' M Eng', 'MEng'),
    (' MEng', 'MEng'),
    (' M.Eng', 'MEng'),
    (' MS', 'MS'),
    (' MA', 'MA'),
    (' Masters', 'MS'),
    (' PhD', 'PhD'),
    (' MBA', 'MBA'),
    (' Other', 'Other'),
    (' EdD', 'Other'),
]

STATUS = {
    'A': 'American',
    'U': 'International with US Degree',
    'I': 'International',
    'O': 'Other',
}

errlog = {'major': [], 'gpa': [], 'general': [], 'subject': []}


def process(index, col):
    global err
    inst, major, degree, season, decision, status, date_add, date_add_ts, comment = None, None, None, None, None, None, None, None, None
    decisionfin, method, decdate, decdate_ts = None, None, None, None
    gpafin, grev, grem, grew, new_gre, sub = None, None, None, None, None, None
    if len(col) != 6:
        Print("Length of column not equal to 6")
    try:
        inst = col[0].text.strip()
    except Exception as e:
        print('starting error ', e)
    try:
        major = None
        progtext = col[1].text.strip()
        if not ',' in progtext:
            print('in major', progtext)
      
            errlog['major'].append((index, col))
        else:
            parts = progtext.split(',')
            major = parts[0].strip()
            progtext = ' '.join(parts[1:])
        degree = None
        for (d, deg) in DEGREE:
            if d in progtext:
                degree = deg
                break
        if not degree:
            degree = 'Other'

        season = None
        mat = re.search('\([SF][01][0-9]\)', progtext)
        if mat:
            season = mat.group()[1:-1]
        else:
            mat = re.search('\(\?\)', progtext)
            if mat:
                season = None
    except NameError  as e:
        print('mat error', e)
    except :
        print("Unexpected error:", sys.exc_info()[0])
    try:
        extra = col[2].find('a',class_='extinfo')
        gpafin, grev, grem, grew, new_gre, sub = None, None, None, None, None, None
        if extra:
            gre_text = extra.text.strip()
            gpa = re.search('Undergrad GPA: ((?:[0-9]\.[0-9]{1,2})|(?:n/a))', gre_text)
            general = re.search('GRE General \(V/Q/W\): ((?:1[0-9]{2}/1[0-9]{2}/(?:(?:[0-6]\.[0-9]{2})|(?:99\.99)|(?:56\.00)))|(?:n/a))',gre_text)
            new_gref = True
            subject = re.search('GRE Subject: ((?:[2-9][0-9]0)|(?:n/a))', gre_text)
            if not gpa:
                gpa = None
            if gpa:
                gpa = gpa.groups(1)[0]
                if not gpa == 'n/a':
                    try:
                        gpafin = float(gpa)
                    except:
                        print("Debug")
            else:
                errlog['gpa'].append((index, gre_text))
            if not general:
                general = re.search(
                    'GRE General \(V/Q/W\): ((?:[2-8][0-9]0/[2-8][0-9]0/(?:(?:[0-6]\.[0-9]{2})|(?:99\.99)|(?:56\.00)))|(?:n/a))',
                    gre_text)
                new_gref = False

            if general:
                general = general.groups(1)[0]
                if not general == 'n/a':
                    try:
                        greparts = general.split('/')
                        if greparts[2] == '99.99' or greparts[2] == '0.00' or greparts[2] == '56.00':
                            grew = None
                        else:
                            grew = float(greparts[2])
                        grev = int(greparts[0])
                        grem = int(greparts[1])
                        new_gre = new_gref
                        if new_gref and (grev > 170 or grev < 130 or grem > 170 or grem < 130 or (
                                grew and (grew < 0 or grew > 6))):
                            errlog['general'].append((index, gre_text))
                            grew, grem, grev, new_gre = None, None, None, None
                        elif not new_gref and (grev > 800 or grev < 200 or grem > 800 or grem < 200 or (
                                grew and (grew < 0 or grew > 6))):
                            errlog['general'].append((index, gre_text))
                            grew, grem, grev, new_gre = None, None, None, None
                    except Exception as e:
                        print("Debug")
            else:
                errlog['general'].append((index, gre_text))
                general = None

            if subject:
                subject = subject.groups(1)[0]
                if not subject == 'n/a':
                    sub = int(subject)
            else:
                errlog['subject'].append((index, gre_text))
                subject = None

            extra.extract()
        decision = col[2].text.strip()
        try:
            #decisionfin, method, decdate, decdate_ts = None, None, None, None
            (decisionfin, method, decdate) = re.search(
                '((?:Accepted)|(?:Rejected)|(?:Wait listed)|(?:Other)|(?:Interview))? ?via ?((?:E-[mM]ail)|(?:Website)|(?:Phone)|(?:Other)|(?:Postal Service)|(?:Unknown))? ?on ?([0-9]{1,2} [A-Z][a-z]{2} [0-9]{4})?',
                decision).groups()
            if decisionfin and decisionfin == 'Accepted':
                decisionfin = 'Accepted'
            elif decisionfin and decisionfin == 'Rejected':
                decisionfin = 'Rejected'
            elif decisionfin and decisionfin == 'Wait listed':
                decisionfin = 'Wait listed'
            elif decisionfin and decisionfin == 'Unknown':
                decisionfin = 'Other'
            elif decisionfin and decisionfin == 'Interview':
                decisionfin = 'Interview'
            else:
                decisionfin = None
            if method and method == 'E-Mail':
                method = 'E-mail'
            if method and method == 'Unknown':
                method = 'Other'
            if decdate:
                try:
                    decdate_date = datetime.datetime.strptime(decdate, '%d %b %Y')
                    decdate_ts = decdate_date.strftime('%s')
                    decdate = (decdate_date.day, decdate_date.month, decdate_date.year)
                except Exception as e:
                    decdate_date, decdate_ts, decdate = None, None, None
            else:
                decdate = None
        except Exception as e:
            print('after decdate', e)
    except Exception as e:
        print('last error',e)
    try:
        statustxt = col[3].text.strip()
        if statustxt in STATUS:
            status = STATUS[statustxt]
        else:
            status = None
    except Exception as e:
        print('status text',e)
    try:
        date_addtxt = col[4].text.strip()
        date_add_date = datetime.datetime.strptime(date_addtxt, '%d %b %Y')
        date_add_ts = date_add_date.strftime('%s')
        date_add = (date_add_date.day, date_add_date.month, date_add_date.year)
    except:
        print(col[4].text.strip()).encode('ascii', 'ignore')
        print("Debug")
    try:
        comment = col[5].text.strip()
    except:
        print("Debug")
    res = [inst, major, degree, season, decisionfin, method, decdate, decdate_ts, gpafin, grev, grem, grew, new_gre,
           sub, status, date_add, date_add_ts, comment]
    return res


data = []
for year in range(1691,1948):
    with open('/html/{0}.html'.format(year), 'r') as f:
        soup = BeautifulSoup(f.read())
        tables = soup.findAll('table', class_='submission-table')
        for tab in tables:
            rows = tab.findAll('tr')
            for row in rows[1:]:
                cols = row.findAll('td')
                pro = process(year, cols)
                if len(pro) > 0:
                    data.append(pro)
        print(year)
        #print(data)
#for d in data:
#    d[1] = d[1].encode('ascii', 'ignore')
df = pandas.DataFrame(data)
df.to_csv('./data/csv/Seventh.csv')

/Users/Sagar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  """


1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890


In [37]:
import pandas as pd


df = pd.read_csv("/csv/First.csv")
df1 = pd.read_csv("/csv/Second.csv")
df2 = pd.read_csv("/csv/Third.csv")
df3 = pd.read_csv("/csv/Fourth.csv")
df4 = pd.read_csv("/csv/Fifth.csv")
df5 = pd.read_csv("/csv/Sixth.csv")
df6 = pd.read_csv("/csv/Seventh.csv")

out = df.append(df1, ignore_index=True)
out = out.append(df2, ignore_index=True)
out = out.append(df3, ignore_index=True)
out = out.append(df4, ignore_index=True)
out = out.append(df5, ignore_index=True)
out = out.append(df6, ignore_index=True)

with open('/csv/Final.csv', 'w', encoding='utf-8') as f:
    out.to_csv(f)
out.shape

(485250, 19)

In [38]:
df = pd.read_csv('/csv/Final.csv')
df.shape

/Users/Sagar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(485250, 20)

In [39]:
df.head()

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,0,University Of Waterloo,Systems Design Engineering,MS,NaN,Accepted,Website,"(1, 7, 2019)",1.561964e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(10, 7, 2019)",1562742000,NaN
1,1,1,Northeastern University,Electrical Engineering,PhD,F19,Rejected,Website,"(8, 7, 2019)",1.562569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, 7, 2019)",1562742000,Got email today!
2,2,2,The University Of Auckland,Electrical And Electronic Engineering,MS,NaN,Accepted,Website,"(19, 6, 2019)",1.560928e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(9, 7, 2019)",1562655600,Top Student\nIELTS: 7.5\nContacted Supervisor\...
3,3,3,Radford University,Counseling Psychology PsyD.,Other,F19,Accepted,Phone,"(4, 3, 2019)",1.551686e+09,NaN,NaN,NaN,NaN,NaN,NaN,American,"(9, 7, 2019)",1562655600,NaN
4,4,4,University Of Chittagong,Computer Science,MS,NaN,NaN,Other,"(9, 7, 2019)",1.562656e+09,3.2,163.0,168.0,4.0,True,NaN,International,"(9, 7, 2019)",1562655600,NaN
